In [1]:
import numpy as np
import math
import gym
from gym import spaces
import time
import random
from stable_baselines3 import A2C
import torch
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
gym.__version__

'0.21.0'

In [4]:
torch.__version__

'1.13.1'

In [5]:
!python3 -V

Python 3.7.7


## Utils

In [6]:
N_DISCRETE_ACTIONS=3

In [7]:
def alter_coord(action, position, g_coord, dx=0.1, change_nodes=[2,3,5,7]):
        
    if action==0:
        g_coord[int(2*change_nodes[position])]+=dx
    elif action==1:
        g_coord[int(2*change_nodes[position])]-=dx
    elif action==2:
        g_coord[int(2*change_nodes[position])]+=0
             
    return g_coord

In [8]:
# this function must be tailored to different FE models
def observe(position, coord,l1,l2, displ):    
    return np.array([position, coord[4]/l2, coord[6]/l1, coord[10]/l1, coord[14]/l2, displ[9]])

In [9]:
def draw(coord,color,elcon):
            coord=coord.reshape(np.max(elcon)+1,2)
            plt.figure(figsize=(13,5))
            for item in elcon:
                plt.plot([coord[item[0]][0],coord[item[1]][0]],[coord[item[0]][1],coord[item[1]][1]],color=color)

            plt.show() 

In [10]:
# this function must be tailored to different FE models
def reward_(obs_,obs):   
    if abs(obs_[5])>abs(obs[5]):  
        return 1
    else:
        return 0

## Finite Element Model of the Plane Truss structure

In [11]:
def PlaneTrussElementLength(x1,y1,x2,y2):
    return math.sqrt((x2-x1)*(x2-x1) + (y2-y1)*(y2-y1))

In [12]:
def PlaneTrussElementStiffness(E,A,L,theta):
    pi=3.14159265
    x = theta*pi/180
    C = math.cos(x)
    S = math.sin(x)
    return E*A/L*np.array([[C*C, C*S, -C*C, -C*S],[C*S, S*S, -C*S, -S*S],
                           [-C*C, -C*S, C*C, C*S],[-C*S, -S*S, C*S, S*S]]) 

In [13]:
def PlaneTrussAssemble(K,k,i,j):
    # K - np.array
    # k - np.array
    K[2*i,2*i] = K[2*i,2*i] + k[0,0]
    K[2*i,2*i+1] = K[2*i,2*i+1] + k[0,1] 
    K[2*i,2*j] = K[2*i,2*j] + k[0,2] 
    K[2*i,2*j+1] = K[2*i,2*j+1] + k[0,3]
    K[2*i+1,2*i] = K[2*i+1,2*i] + k[1,0]
    K[2*i+1,2*i+1] = K[2*i+1,2*i+1] + k[1,1] 
    K[2*i+1,2*j] = K[2*i+1,2*j] + k[1,2] 
    K[2*i+1,2*j+1] = K[2*i+1,2*j+1] + k[1,3] 
    K[2*j,2*i] = K[2*j,2*i] + k[2,0] 
    K[2*j,2*i+1] = K[2*j,2*i+1] + k[2,1]
    K[2*j,2*j] = K[2*j,2*j] + k[2,2]
    K[2*j,2*j+1] = K[2*j,2*j+1] + k[2,3] 
    K[2*j+1,2*i] = K[2*j+1,2*i] + k[3,0] 
    K[2*j+1,2*i+1] = K[2*j+1,2*i+1] + k[3,1]
    K[2*j+1,2*j] = K[2*j+1,2*j] + k[3,2] 
    K[2*j+1,2*j+1] = K[2*j+1,2*j+1] + k[3,3]
    
    return K   

In [14]:
def PlaneTrussElementForce(E,A,L,theta,u):
    pi=3.14159265
    x = theta* pi/180
    C = math.cos(x)
    S = math.sin(x)
    return E*A/L*np.dot(np.array([-C, -S, C, S]),u)

In [15]:
def PlaneTrussElementStress(E,L,theta,u):
    pi=3.14159265
    x = theta * pi/180
    C = math.cos(x)
    S = math.sin(x)
    return E/L*np.dot(np.array([-C, -S, C, S]),u)

In [16]:
def FEA_u(coord, elcon, bc_u_elim, f_after_u_elim, A=1, E=2e5):
    K=np.zeros(shape=(2*np.max(elcon)+2,2*np.max(elcon)+2))
    pi=3.14159265
    for el in elcon:
        L=PlaneTrussElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
        theta=math.atan((coord[el[1]][1]-coord[el[0]][1])/(coord[el[1]][0]-coord[el[0]][0]+1e-13))*180/pi
        k=PlaneTrussElementStiffness(E,A,L,theta)
        K=PlaneTrussAssemble(K,k,el[0],el[1])
    K=np.delete(K,bc_u_elim,0)
    K=np.delete(K,bc_u_elim,1)
    
    
    d=np.linalg.solve(K, f_after_u_elim)

        
    u=np.zeros(shape=(2*len(coord)))

    j=0
    for i in range(len(u)):
        if i not in bc_u_elim:
            u[i]=d[j]
            j+=1
            if j>len(d)-1:
                break
    return u

In [17]:
class FemEnv(gym.Env):
    
    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super().__init__()
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        self.pst=random.randint(0,3)
        self.g_coord = alter_coord(2, self.pst, np.array([0.0,0.0,2.0,3.0,2.0,0.0,5.0,3.0,7.0,0.0,9.0,3.0,12.0,3.0,12.0,0.0,14.0,0.0]),
                                          dx=0.1, change_nodes=[2,3,5,7])
        self.displ = FEA_u(self.g_coord.reshape(9,2), elcon=np.array([[0, 1],[0, 2],[1, 2],[1, 3],[2, 3],
                                                    [2, 4],[3, 4],[3, 5],[4, 5],[4, 7],
                                                    [5, 7],[5, 6],[6, 7],[6, 8],[7, 8]]),
                                                    bc_u_elim=[0,1,16,17],
                                                    f_after_u_elim=np.array([0,0,0,0,0,0,0,-10,0,0,0,0,0,0]), 
                                                    A=1, E=2e5)
        
        self.obs=observe(self.pst, self.g_coord, 10, 14, self.displ)
        
        self.observation_space = spaces.Box(low=np.array([-np.inf,
                                                          -np.inf,
                                                          -np.inf,
                                                          -np.inf,
                                                          -np.inf,
                                                          -np.inf]),
                                            high=np.array([np.inf,
                                                           np.inf,
                                                           np.inf,
                                                           np.inf,
                                                           np.inf,
                                                           np.inf]),shape=(6,),
                                           dtype=np.float64)

    def step(self, action):
        obs_=self.obs 
        
        g_coord = alter_coord(action, self.pst, self.g_coord, dx=0.1, change_nodes=[2,3,5,7])
        
        pst=random.randint(0,3)
        self.pst=pst
        
        done=False
        
        if g_coord[4]<g_coord[0]+0.2 or g_coord[4]>g_coord[8]-0.2:
            done=True

        if g_coord[6]<g_coord[2]+0.2 or g_coord[6]>g_coord[12]-0.2:
            done=True

        if g_coord[10]<g_coord[2]+0.2 or g_coord[10]>g_coord[12]-0.2:
            done=True

        if g_coord[14]<g_coord[8]+0.2 or g_coord[14]>g_coord[16]-0.2:
            done=True   

        displ = FEA_u(g_coord.reshape(9,2), elcon=np.array([[0, 1],[0, 2],[1, 2],[1, 3],[2, 3],
                                            [2, 4],[3, 4],[3, 5],[4, 5],[4, 7],
                                            [5, 7],[5, 6],[6, 7],[6, 8],[7, 8]]),
                                            bc_u_elim=[0,1,16,17],
                                            f_after_u_elim=np.array([0,0,0,0,0,0,0,-10,0,0,0,0,0,0]), 
                                            A=1, E=2e5)
        self.obs=observe(pst,g_coord,10,14,displ)               
        reward=reward_(obs_,self.obs)
        self.g_coord=g_coord
      
        return self.obs, reward, done, dict()

    def reset(self):
        self.pst = random.randint(0,3)
        self.g_coord = alter_coord(2, self.pst, np.array([0.0,0.0,2.0,3.0,2.0,0.0,5.0,3.0,7.0,0.0,9.0,3.0,12.0,3.0,12.0,0.0,14.0,0.0]),
                                          dx=0.1, change_nodes=[2,3,5,7])
        self.displ = FEA_u(self.g_coord.reshape(9,2), elcon=np.array([[0, 1],[0, 2],[1, 2],[1, 3],[2, 3],
                                                    [2, 4],[3, 4],[3, 5],[4, 5],[4, 7],
                                                    [5, 7],[5, 6],[6, 7],[6, 8],[7, 8]]),
                                                    bc_u_elim=[0,1,16,17],
                                                    f_after_u_elim=np.array([0,0,0,0,0,0,0,-10,0,0,0,0,0,0]), 
                                                    A=1, E=2e5)
        
        self.obs=observe(self.pst, self.g_coord, 10, 14, self.displ)
        return self.obs  

    def render(self, mode="human"):
        
        
        draw(self.g_coord, color="green", elcon=np.array([[0, 1],[0, 2],[1, 2],[1, 3],[2, 3],
                                                    [2, 4],[3, 4],[3, 5],[4, 5],[4, 7],
                                                    [5, 7],[5, 6],[6, 7],[6, 8],[7, 8]]))    

    def close(self):
        pass

In [18]:
from stable_baselines3.common.env_checker import check_env

env = FemEnv()
check_env(env)

In [ ]:
start=time.time()
model = A2C("MlpPolicy", env).learn(total_timesteps=500000)
end=time.time()   

In [ ]:
print('Total time taken: {}'.format(end - start))

### Design by AI

In [ ]:
obs = env.reset()
i=0
while i<50:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    i+=1
    
env.render()    

In [ ]:
obs[-1]

In [ ]:
# Benchmark from initial design:
-0.8888E-03

### Initial Design 

In [ ]:
draw(np.array([0.0,0.0,2.0,3.0,2.0,0.0,5.0,3.0,7.0,0.0,9.0,3.0,12.0,3.0,12.0,0.0,14.0,0.0]),
     color="blue",
     elcon=np.array([[0, 1],[0, 2],[1, 2],[1, 3],[2, 3],
                    [2, 4],[3, 4],[3, 5],[4, 5],[4, 7],
                    [5, 7],[5, 6],[6, 7],[6, 8],[7, 8]]))